# Previsão de categorias com K-Nearest Neighbors

**Objetivo**: O objetivo deste notebook é prever se uma transação é fraudulenta ou não usando o algoritmo K-NN com scikit-learn.

## Índice

1. Preparação de dados
2. Implementação do algoritmo k-NN
3. Ajuste de parâmetros usando GridsearchCV
4. Dimensionamento

## Requisitos e pacote

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

## Preparação de dados

In [ ]:
#Reading in the dataset

df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [ ]:
#Viewing the data

df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


**Eliminando os recursos redundantes**

In [ ]:
#Dropping the redundant features

df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

In [ ]:
#Inspecting the data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 8 columns):
step              int64
type              object
amount            float64
oldbalanceOrg     float64
newbalanceOrig    float64
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
dtypes: float64(5), int64(2), object(1)
memory usage: 388.3+ MB


**Reduzindo o tamanho dos dados**

In [ ]:
#Storing the fraudulent data into a dataframe

df_fraud = df[df['isFraud'] == 1]

In [ ]:
#Storing the non-fraudulent data into a dataframe 

df_nofraud = df[df['isFraud'] == 0]

In [ ]:
#Storing 12,000 rows of non-fraudulent data

df_nofraud = df_nofraud.head(12000)

In [ ]:
#Joining both datasets together 

df = pd.concat([df_fraud, df_nofraud], axis = 0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20213 entries, 2 to 12071
Data columns (total 8 columns):
step              20213 non-null int64
type              20213 non-null object
amount            20213 non-null float64
oldbalanceOrg     20213 non-null float64
newbalanceOrig    20213 non-null float64
oldbalanceDest    20213 non-null float64
newbalanceDest    20213 non-null float64
isFraud           20213 non-null int64
dtypes: float64(5), int64(2), object(1)
memory usage: 1.4+ MB


**Codificando o recurso categórico**

In [ ]:
#Converting the type column to categorical

df['type'] = df['type'].astype('category')

In [ ]:
#Integer Encoding the 'type' column

type_encode = LabelEncoder()

In [ ]:
#Integer encoding the 'type' column

df['type'] = type_encode.fit_transform(df.type)

In [ ]:
df['type'].value_counts()

3    6732
1    5694
4    5231
0    2186
2     370
Name: type, dtype: int64

In [ ]:
#One hot encoding the 'type' column

type_one_hot = OneHotEncoder()
type_one_hot_encode = type_one_hot.fit_transform(df.type.values.reshape(-1,1)).toarray()

In [ ]:
#Adding the one hot encoded variables to the dataset 

ohe_variable = pd.DataFrame(type_one_hot_encode, columns = ["type_"+str(int(i)) for i in range(type_one_hot_encode.shape[1])])
df = pd.concat([df, ohe_variable], axis=1)

In [ ]:
#Dropping the original type variable 

df = df.drop('type', axis = 1)

**Verificando valores ausentes**

In [ ]:
#Checking every column for missing values

df.isnull().any()

step              True
type              True
amount            True
oldbalanceOrg     True
newbalanceOrig    True
oldbalanceDest    True
newbalanceDest    True
isFraud           True
type_0            True
type_1            True
type_2            True
type_3            True
type_4            True
dtype: bool

In [ ]:
#Imputing the missing values with a 0

df = df.fillna(0)

In [ ]:
#Checking if there are missing values left

df.isnull().any()

step              False
type              False
amount            False
oldbalanceOrg     False
newbalanceOrig    False
oldbalanceDest    False
newbalanceDest    False
isFraud           False
type_0            False
type_1            False
type_2            False
type_3            False
type_4            False
dtype: bool

**Exportando o conjunto de dados**

In [ ]:
df.to_csv('fraud_prediction.csv')

## Implementando o Algoritmo k-NN

In [ ]:
#Creating the features 

features = df.drop('isFraud', axis = 1).values
target = df['isFraud'].values

**Dividindo os dados em conjuntos de treinamento e teste**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42, stratify = target)

**Construindo o classificador knn**

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [ ]:
knn_classifier.score(X_test, y_test)

0.98306679209783632

## Parâmetros de ajuste fino usando GridSearchCV

In [ ]:
#Initializing a grid with possible number of neighbors from 1 to 24

grid = {'n_neighbors' : np.arange(1, 25)}

#Initializing a k-NN classifier 

knn_classifier = KNeighborsClassifier()

#Using cross validation to find optimal number of neighbors 

knn = GridSearchCV(knn_classifier, grid, cv = 10)

knn.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
#Extracting the optimal number of neighbors 

knn.best_params_

{'n_neighbors': 1}

In [ ]:
#Extracting the accuracy score for optimal number of neighbors

knn.best_score_

0.9850813971070006

## Scaling

In [ ]:
#Setting up the scaling pipeline 

pipeline_order = [('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_neighbors = 1))]

pipeline = Pipeline(pipeline_order)

#Fitting the classfier to the scaled dataset 

knn_classifier_scaled = pipeline.fit(X_train, y_train)

#Extracting the score 

knn_classifier_scaled.score(X_test, y_test)

0.99753057384760113